In [ ]:
# General libraries
import pandas as pd
import numpy as np
from collections import defaultdict
import random
from random import randrange
import math

In [ ]:
# dictionaries with the structure source -> array of followings
dictionary = defaultdict(list)
# simple datafram to store the sources and the count of source followings
data = pd.DataFrame([])

file = open('data/train.txt', 'r') 
lines = file.readlines() 
count = 0
for line in lines: 
    count = count + 1
    split_string = list(map(int,line.strip().split("\t")))
    dictionary[split_string[0]] = []
    if (len(split_string) >1):
        dictionary[split_string[0]] = sorted(split_string[1:len(split_string)])
        
    data = data.append(pd.DataFrame({'Source': split_string[0], 'Source_Followings': len(dictionary[split_string[0]]) }, index=[0]), ignore_index=True)


In [ ]:
data.tail(10)

In [ ]:
# function to calculate followers
def calculateFollowers(data,node):
    followers = 0
    for j in range(0,19999):
        if node in dictionary[data.at[j,'Source']]:
            followers = followers + 1
    
    return followers

# Second version: similar performance
# function to calculate followers
def calculateFollowers2(data,node):
    followers = 0
    for j in range(0,19999):
        common_elements = set(dictionary[data.at[j,'Source']]).intersection([node]) 
        followers = followers + len(common_elements)
    
    return followers

# Test
calculateFollowers(data,4778890)

In [ ]:
# REAL EDGES
# First version
# Random selection of 2000 real edges
real_edges = pd.DataFrame([])
i = 0
while i < 2000:
    if i % 100 == 0:
        print(i)
    x = randrange(0, 19999)
    source = data.at[x,'Source']
    source_followings = data.at[x,'Source_Followings']
    if (source_followings > 0):
        i += 1
        y = random.randint(0, source_followings-1)
        sink = dictionary[source][y]
        source_followers = calculateFollowers(data,source)
        sink_followers = calculateFollowers(data,sink)
        real_edges = real_edges.append(pd.DataFrame({'Source': source,'Sink': sink,
                                                     'Source_Followings': source_followings,
                                                     'Source_Followers': source_followers,
                                                     'Sink_Followers': sink_followers},
                                                    index=[0]), ignore_index=True)

In [ ]:
# REAL EDGES
# Second version
# Random walk with jump (probability 0.1) for creating 4000 real edges
jump_probability = 0.1
real_edges = pd.DataFrame(columns=['Source','Sink','Source_Followings','Source_Followers','Sink_Followers'])
#seed
source = 540762
changeSource = False

i = 0
while i < 4000:
    if i % 100 == 0:
        print(i)
        
    if (data.loc[(data['Source'] == source)].any().values[0]):
        source_followings = data.loc[data['Source'] == source, 'Source_Followings'].values[0]
        if (source_followings > 0):
            changeSource = False
            y = random.randint(0, source_followings-1)
            sink = dictionary[source][y]
            # it is checked if the pair is not already added
            if not(real_edges.loc[(real_edges['Source'] == source) & (real_edges['Sink'] == sink)].any().values[0]):
                i += 1
                source_followers = calculateFollowers(data,source)
                sink_followers = calculateFollowers(data,sink)
                real_edges = real_edges.append(pd.DataFrame({'Source': source,'Sink': sink,
                                                             'Source_Followings': source_followings,
                                                             'Source_Followers': source_followers,
                                                             'Sink_Followers': sink_followers},
                                                           index=[0]), ignore_index=True)
                source = sink
    else:
        changeSource = True
      
    jump = np.random.rand() < jump_probability
    if jump or changeSource:
        x = randrange(0, 19999)
        source = data.at[x,'Source'] 

In [ ]:
real_edges.head(10)

In [ ]:
real_edges.to_csv("data_generated/real_edges.csv", sep='\t', index=False)

In [ ]:
## FAKE EDGES
# First version
# Random creation of 4000 fake edges

fake_edges = pd.DataFrame([])
i = 0
while i < 4000:
    if i % 100 == 0:
        print(i)
    
    x = randrange(0, 19999)
    source = data.at[x,'Source']
    source_followings = data.at[x,'Source_Followings']
    
    y = randrange(0, 19999)
    source_gen = data.at[y,'Source']
    source_gen_followings = data.at[y,'Source_Followings']
    
    if (source_gen_followings > 0):
        z = random.randint(0, source_gen_followings-1)
        sink = dictionary[source_gen][z]

        # It is not real, is added
        if sink not in dictionary[source]:
            i += 1
            source_followers = calculateFollowers(data,source)
            sink_followers = calculateFollowers(data,sink)
            fake_edges = fake_edges.append(pd.DataFrame({'Source': source,'Sink': sink,
                                                         'Source_Followings': source_followings,
                                                         'Source_Followers': source_followers,
                                                         'Sink_Followers': sink_followers},
                                                        index=[0]), ignore_index=True)

In [ ]:
## FAKE EDGES
# Second version 
# random creation of 4000 fake edges
jump_probability = 0.5
nextNode = True
jump = True

fake_edges = pd.DataFrame([])
i = 0
while i < 4000:
    
    if i % 100 == 0:
        print(i)
        
    changeSource = False
    jump = True
    x = randrange(0, 19999)
    source = data.at[x,'Source']
    source_followings = data.loc[data['Source'] == source, 'Source_Followings'].values[0]

    if (source_followings > 0): 
        y = random.randint(0, source_followings-1)
        next_source = dictionary[source][y]
 
        while jump and not changeSource:
            
            if (data.loc[(data['Source'] == next_source)].any().values[0]):
                next_source_followings = data.loc[data['Source'] == next_source, 'Source_Followings'].values[0]
            else:
                next_source_followings = 0
                changeSource = True
                
            if (next_source_followings > 0):
  
                z = random.randint(0, next_source_followings-1)
                sink = dictionary[next_source][z]
                jump = np.random.rand() < jump_probability

                if jump:
                    next_source = sink 
                else:
                    changeSource = True
                    # It is not real, it is added
                    if sink not in dictionary[source]:
                        i += 1
                        source_followers = calculateFollowers(data,source)
                        sink_followers = calculateFollowers(data,sink)
                        fake_edges = fake_edges.append(pd.DataFrame({'Source': source,'Sink': sink,
                                                                     'Source_Followings': source_followings,
                                                                     'Source_Followers': source_followers,
                                                                     'Sink_Followers': sink_followers},
                                                                    index=[0]), ignore_index=True) 
            else:
                changeSource = True

In [ ]:
fake_edges.head()

In [ ]:
fake_edges.to_csv("data_generated/fake_edges4.csv", sep='\t', index=False)

In [ ]:
# Calculation of features for test data
test_data = pd.DataFrame([])
for index, row in test_public.iterrows():
    if index % 100 == 0:
        print(index)
    source = row['Source']
    source_followings = data.loc[data['Source'] == source, 'Source_Followings'].iloc[0]
    sink = row['Sink']

    source_followers = calculateFollowers(data,source)
    sink_followers = calculateFollowers(data,sink)




    test_data = test_data.append(pd.DataFrame({'Id': row['Id'], 'Source': source,'Sink': sink,
                                                 'Source_Followings': source_followings,
                                                 'Source_Followers': source_followers,
                                                 'Sink_Followers': sink_followers},
                                                index=[0]), ignore_index=True) 

In [ ]:
test_data.to_csv("data_generated/test_data.csv", sep='\t', index=False)

End

# 